In [1]:
from collections     import  Counter
from itertools       import product
from typing          import List, Generator
from helpers         import data

In [2]:
start = data(17)
start

['#.#####.',
 '#..##...',
 '.##..#..',
 '#.##.###',
 '.#.#.#..',
 '#.##..#.',
 '#####..#',
 '..#.#.##']

In [3]:
len(start), len(start[0])

(8, 8)

A **neighbor** is any cube where **any coordinate differs by at most 1**. 

Active cubes are denoted with \#, and inactive cubes are denoted with \. 

The boot process consists of six cycles. During each cycle: 
- If a cube is **active** and exactly **2 or 3 neighbors are active**, it stays active. Otherwise it becomes inactive. 
- If a cube is **inactive** and exactly **3 neighors are active**, it becomes active. Otherwise it stays inactive. 

**Part 1:** How many cubes are left in the active state after the sixth cycle?

In [4]:
Cube = tuple

def parse_image(image: List[str], active="#", d=3) -> set[Cube]:
    """Parse image as a starting point for d-dimensional Conway game."""
    return {(x, y, *(0,) * (d-2)) 
             for y in range(len(image))
             for x, c in enumerate(image[y]) 
             if c == active}

In [5]:
def simulate(cubes: set[Cube]) -> set[Cube]:
    """Simulate a time step according to set rules.
    TODO: Rules should be changeable!
    """
    n_neighbors = count_neighbors(cubes)
    return {neighbor 
            for neighbor, n in n_neighbors.items()
            if n == 3 
            or n == 2 and neighbor in cubes}

def offsets(d: int) -> Generator[tuple, None, None]:
    """Generate offsets to apply to each cell."""
    # I had to peek at Norvig's code again because I remembered he did something 
    # similar, and I was having a bug. The problem was I forgot to exclude the 
    # (0, 0, 0) offset via any. 
    yield from filter(any, product((-1, 0, 1), repeat=d))

def count_neighbors(cubes: set[Cube]) -> dict[Cube, int]:
    """Generate mapping from Cube to number of neighbors."""       
    n_neighbors = Counter()
    for cube in cubes: 
        # Add each active cube's neighbors 
        for offset in offsets(len(cube)):
            neighbor = tuple(cube[i] + offset[i] for i in range(len(cube)))
            n_neighbors[neighbor] += 1
    return n_neighbors

In [6]:
def life(cubes: set[Cube], steps: int) -> set[Cube]:
    """Simulate for `steps` iterations."""
    for step in range(steps): 
        cubes = simulate(cubes)
    return cubes

def count_active(cubes: set[Cube]) -> int: 
    return len(cubes)

In [7]:
# Active cubes
cubes = parse_image(start)
assert (0, 0, 0) in cubes

count_active(life(cubes, steps=6))

353

**Part 2:** Simulate 6 steps in a 4D space. 

In [8]:
cubes = parse_image(start, d=4)

count_active(life(cubes, steps=6))

2472

**Norvig:** The biggest thing I'm taking away is that there's always a simpler way to set up a problem... I'm also surprised at how well he knows `itertools`. Another small trick is to use `Counter` instead of a `defaultdict` that I increment.

In [9]:
def count_neighbors(cubes: set[Cube]) -> dict[Cube, int]:
    """Generate mapping from Cube to number of neighbors."""       
    return Counter(tuple( map(add, cube, offset) )
                   for cube in cubes
                   for offset in offsets(len(cube)))